In [1]:
import pandas as pd
import numpy as np

import re
import torch
import faiss

from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModel, AutoModelForSeq2SeqLM

In [2]:
df = pd.read_csv('./df_data.csv')
df

,text
0,\nКапитан сборной Аргентины Лионель Месси сооб...
1,\nЛидер сборной Аргентины Лионель Месси вряд л...
2,\nINTERFAX.RU - Аргентинский футболист Лионель...
3,\nЛионель Месси не планирует участвовать в чем...
4,\nКапитан сборной Аргентины по футболу Лионель...
...,...
95,\nТинькофф Банк объявил о запуске Центрального...
96,\n«Тинькофф» запускает частный университет со ...
97,\n14 июня 2023 года «Тинькофф» объявил о запус...
98,\nТинькофф банк запустит частный университет -...


In [3]:
# Задаем путь к предобученной модели

model_ckpt = "papluca/xlm-roberta-base-language-detection"

In [4]:
# Инициализируем токенизатор и модель, а также функции, которые возвращают эмбеддинги

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)
print(end='')

Some weights of the model checkpoint at papluca/xlm-roberta-base-language-detection were not used when initializing XLMRobertaModel: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaModel were not initialized from the model checkpoint at papluca/xlm-roberta-base-language-detection and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use 

In [20]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]


def get_embeddings(text_list):
    encoded_input = tokenizer(text_list, padding=True, 
                              truncation=True, return_tensors='pt')
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    
    with torch.no_grad():
        model_output = model(**encoded_input)
    
    return cls_pooling(model_output)


# Посчитаем эмбединги в специальной структуре
# Определять 'близость' запроса и темы из датасета будем по эмбеддингам тэгов 

ds = Dataset.from_pandas(df)
embeddings_dataset = ds.map(
    lambda x: {'text_embeddings': get_embeddings(x['text']).detach().cpu().numpy()[0]})
embeddings_dataset.add_faiss_index(column='text_embeddings')

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['text', 'text_embeddings'],
    num_rows: 100
})

In [32]:
# Создадим функцию, которая вернет 'близкие' 10 текстов

def get_sequences(question):
    question_embedding = get_embeddings([question]).cpu().detach().numpy()
    scores, samples = embeddings_dataset.get_nearest_examples(
        'text_embeddings', question_embedding, k=10)
    return scores, samples['text']

In [33]:
# Зададим вопрос и получим упорядоченный список от наиболее вероятных к наименее

question = 'падение криптовалюты'
base_articles = get_sequences(question)

In [34]:
base_articles

(array([20.723663, 23.290821, 23.406754, 24.162502, 24.224155, 24.259247,
        24.577507, 25.397358, 25.462276, 26.063744], dtype=float32),
 ['\nЛионель Месси не планирует участвовать в чемпионате мира-2026.\n\nФорвард сборной Аргентины ответил на вопрос о том, сыграет ли он на следующем мировом первенстве после победы на ЧМ-2022. \n\n«Я думаю, что нет. Это был мой последний чемпионат мира. Я посмотрю, как все сложится, но по идее нет, я не поеду на следующий чемпионат мира», – сказал Месси в интервью Titan Sports.\n',
  '\n«Тинькофф» объявил о запуске своего университета. Поначалу он будет специализироваться на математике и компьютерных науках. Первая программа бакалавриата начнется в сентябре 2024 года, на нее планируют взять 500 студентов и 55 преподавателей. Обучение будет стоить от 70 000 рублей за месяц\n«Тинькофф» создаст собственный вуз — Центральный университет. Об этом объявил председатель правления компании Станислав Близнюк, говорится в сообщении пресс-службы, поступивше

In [51]:
type(base_articles[0])

str

In [33]:
tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/mT5_multilingual_XLSum", use_fast=False)

model = AutoModelForSeq2SeqLM.from_pretrained("csebuetnlp/mT5_multilingual_XLSum")

In [35]:
WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

article_text = '''
Интернет-проект Colta.ru отказался от инвестиционной модели и превратился в общественное СМИ. Об этом "Ленте.ру" рассказала \
главный редактор "Кольты" Мария Степанова. "Наша ситуация - особенная, - рассказала Степанова. - Это значит, что у нас нету \
инвестора, нету собственника, сайт не принадлежит никому, кроме своей редакции и своих читателей, и финансируется\ 
исключительно обществом". На сегодняшний день редакции удалось собрать около 75 процентов годового бюджета сайта. Эти деньги,\
по словам Степановой, "Кольте" предоставили люди, которые войдут в попечительский совет сайта. Численность и состав совета,\
а также его функции, будут названы позже (на сайте 14 января появилась информация, что в состав совета может войти любой\
человек, членство обойдется в 300 тысяч рублей в год). Оставшиеся деньги - 2 миллиона 700 тысяч рублей - редакция\
намеревается искать самостоятельно. Первый транш, 400 тысяч рублей, "Кольта" надеется собрать с помощью краудфандинга на\
сайте "Планета.ру" - летом 2012 года с помощью этой платформы Colta.ru собрала 700 тысяч рублей, частично обеспечивших первые\
месяцы работы сайта вплоть до закрытия. На новой странице на "Планете" предусмотрены лоты от 100 рублей до искомых 2,7\
миллиона. По словам Степановой, столько же понадобится "Кольте" и на будущий год: 70-75 процентов от годового бюджета редакция\
вновь получит от попечителей. Сайт Colta.ru, запущенный командой старого OpenSpace в июле 2012 года и закрывшийся в ноябре\
из-за отсутствия инвесторов, возобновил работу 14 января 2013 года. Как рассказала Мария Степанова, первое время дизайн\
"Кольты" останется неизменным, хотя редакция планирует увеличить объем ежедневных обновлений по основным темам. Кроме того,\
14 января было объявлено, что Colta.ru станет партнером издания Slon.ru - некоторые материалы "Кольты" будут доступны на сайте\
"Слона". В марте 2013 года Colta.ru планирует провести перезапуск - в новом дизайне сайт станет больше похожа на старый\
OpenSpace, в частности вернутся тематические рубрики, которые в текущей версии "Кольты" скрыты. Кроме того, обновленный сайт\
предложит читателям подписку и другие удобные варианты оказания финансовой помощи "Кольте".
'''

input_ids = tokenizer(
    [WHITESPACE_HANDLER(article_text)],
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=1024
)["input_ids"]

output_ids = model.generate(
    input_ids=input_ids,
    min_length=400,
#     max_length=1000,
    no_repeat_ngram_size=3,
    num_beams=4,
    max_new_tokens=500
)[0]

summary = tokenizer.decode(
    output_ids,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=False
)

print(summary)

Both `max_new_tokens` (=500) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Российский интернет-проект Colta.ru, закрывшийся в ноябре 2012 года из-за отсутствия инвестора, вновь возобновил работу. Редакция надеется собрать около 75 процентов годового бюджета сайта и получить деньги от попечительского совета. Это означает, что сайт не принадлежит никому.  Так и считает редакция "Кольты", которая отказалась от инвестиционной модели и превратилась в общественное СМИ, а в этом году - .    Эти и.о. "Ленте.ru" - "Кольте" уже давно пытается купить деньги, которые, по ее словам, будет работать уже в новом дизайне, однако в ближайшее время может оказаться одной из самых популярных в российском сегменте соцсетей, и это может привести к тому, что его редакция будет получать деньги только от инвесторов и своих читателей. Что будет с ним в следующем году и как можно скорее, чтобы получить финансовую помощь для создания новой версии сайта, которая была запущена в 2011 году, когда компания OpenSpace была закрыта и закрылась, но теперь ее создатели не могут работать с инвесто